In [1]:
import glob
import pyrender
import trimesh
import os
from typing import Union,List,Dict,Tuple
import re
import numpy as np
from pickle import load,dump
folder = "/local/home/lrahmann/Comparison/data/nglod/"
files = glob.glob(f"{folder}/*.obj") + glob.glob(f"{folder}/*.off") 

In [2]:
#import sys
#sys.path.append("..")
#from computeChamfer import Compute_Chamfer

In [3]:
class Config:
    basefolder : str = "."
    bg_color = [0.25,0.25,0.25,0]
    ambient_light = [0.5,0.5,0.5]
    @staticmethod
    def Viewer(scene:pyrender.Scene) -> pyrender.Viewer:
        return pyrender.Viewer(scene,render_flags={"all_solid":True},viewer_flags={"use_direct_lighting":True})
    def Scene() -> pyrender.Scene:
        return pyrender.Scene(bg_color = Config.bg_color, ambient_light = Config.ambient_light)
        

In [4]:
class Mesh(object):
    """A small wrapper for meshes to provide tools to compute the chamfer distance etc. and rendering over pyrenderer
    
    Attributes:
        filename (str): Filename used for importing/exporting the mesh
        translation (np.ndarray): Translation of the mesh e.g. for normalization
        scale (np.ndarray): scale of the mesh per axis
        children (List[Mesh]): list of the children meshes attributed with this mesh (e.g. chamfer distance)
        values : Dict[str,float] 
    """
    filename : str = None
    mesh :  pyrender.mesh.Mesh = None
    translation : np.ndarray = np.array([0.0,0.0,0.0])
    name : str = None
    scale : np.ndarray = np.array([1.0,1.0,1.0])
        
    children : List["Mesh"] = None
    values : Dict[str,float] = None
    
    def __init__(self, name : str, 
                 filename : str = None, 
                 vertices : np.ndarray = None, 
                 faces : np.ndarray = None, 
                 normals : np.ndarray = None,
                 colors : np.ndarray = None):
        self.filename = filename
        self.name = name
        self.children = []
        values = {}
        if vertices:
            if not faces:
                self.mesh = pyrender.from_points(vertices,normals,colors)
            else:
                self.mesh = pyrender.Mesh.from_trimesh(trimesh.Trimesh(vertices,faces))
                
    
    def load(self):
        if not self.mesh:
            mesh = trimesh.load_mesh(self.filename)
            mesh.faces = mesh.faces[:,::-1]
            self.mesh = pyrender.Mesh.from_trimesh(mesh,smooth = False)
   
    def chamfer(self, other : 'Mesh'):
        pass

    def normalize(self):
        assert(len(self.mesh.primitives) == 1)
        vertices = (self.mesh.primitives[0].positions + self.translation ) * self.scale
        cordmax = vertices.max(axis=0)
        cordmin = vertices.min(axis=0)
        mean = (cordmax+cordmin)/2
        vertices -= mean
        self.translation -= mean
        self.scale /= np.linalg.norm(vertices,axis=1).max()
    
    def show(self):
        pass
    
    def getnode(self):
        return pyrender.Node(name=self.name,scale=self.scale, translation=self.translation)
    
        

In [12]:
class CameraConfiguration:
    
    configurations : Dict[str,'CameraConfiguration'] = {}
    
    pose : np.ndarray = np.eye(4)
    yfov: float = np.pi / 3.0
    ratio: float = 16.0/9.0
    name : str = None
    
    @staticmethod
    def extract_from_viewer(viewer : pyrender.Viewer, config : "CameraConfiguration" = None) -> 'CameraConfiguration':
        if not config:
            config = CameraConfiguration()
        config.pose = viewer._camera_node.matrix.copy()
        config.yfov = viewer._camera_node.camera._yfov
        config.ratio = viewer.viewport_size[0]/viewer.viewport_size[1]
        return config
    
    @staticmethod
    def get_camera_file(self, name:str):
        return f"{Config.basefolder}/{self.name}_{name}_camera.pkl"
   
    @staticmethod
    def load_all():
        g_path = CameraConfiguration.get_camera_file("*")
        g_regex = re.escape(self.get_camera_file("REPLACEME")).replace("REPLACEME","([A-z]*)")
        pattern = re.compile(g_regex)
        for camera in glob.glob(g_path):
            try:
                match = pattern.search(camera)
                if(len(match.group())>0):
                    with open(camera, 'rb') as infile:
                        c = CameraConfiguration()
                        c.__dict__.update(load(infile))
                        CameraConfiguration.configurations[match.group(1)] = c
            except:
                pass
            
    @staticmethod
    def save_all():
        for camera in CameraConfiguration.configurations:
            camera.export()
    
    @staticmethod
    def set_camera(name : str, reference: Mesh, height: int = 720):
        scene : pyrender.Scene = None
        width : int = int(height * (16.0/9.0))
        old_camera : "Camera" = None
        
        if name in CameraConfiguration.configurations:
            camera = CameraConfiguration.configurations[name]
            scene = camera.get_scene()
            width = int(camera.ratio * height)
            old_camera = camera
        else:
            scene = Config.Scene()
            
        scene.add(reference.mesh,reference.name)
        viewer = pyrender.Viewer(scene,viewport_size=(width,height),render_flags={"all_solid":True},viewer_flags={"use_direct_lighting":True})
        cam = CameraConfiguration.extract_from_viewer(viewer,old_camera)
        
        cam.name = name
        if cam not in CameraConfiguration.configurations:
            CameraConfiguration.configurations[name]=cam
        
        print(f"Modified/Added camera with name {name}")
    
    
    def get_scene(self):
        scene = Config.Scene()
        cam = pyrender.PerspectiveCamera(yfov=self.yfov)
        cam_node = scene.add(cam, pose=self.pose)
        scene.main_camera_node = cam_node
        return scene
    
    def reload(self):
        path : str = CameraConfiguration.get_camera_file(self.name)
        with open(path, 'rb') as infile:
            self.__dict__.update(load(infile))
            print(f"Reloaded camera {self.name} filename {path}")
            

    def export(self):
        path : str = CameraConfiguration.get_camera_file(self.name)
        with open(path, 'wb') as outfile:
            dump(cam.__dict__,outfile)
            print(f"Saved camera {self.name} filename {path}")
            
    
    def rm_camera(self, name:str):
        del self.scenes[name]

In [13]:
class Experiment:
    """Describes the set of one experiment (e.g. different methods using the same groundTruth) and compares those with each other
    Attributes:
        name (str): name to use for this experiment in the generated tables/models
        evaluations List[Mesh]: resulting meshes of this experiment
        groundtruth Mesh: ground truth mesh of this experiment
        scenes : camera poses to render
    """
    name : str = None
    evaluations : List[Mesh] = None
    groundtruth : Mesh = None
    scenes : List[CameraConfiguration] = None
    
    def __init__(self, name : str , groundtruth : Mesh):
        self.evaluations = []
        self.scenes = []
        self.groundtruth = groundtruth
        self.name = name 
    
   

In [14]:
def getfilename(path:str)->str:
    return os.path.splitext(os.path.basename(path))[0]
groundTruth = Mesh(getfilename(files[0]),files[0])
groundTruth.load()
groundTruth.normalize()
ex = Experiment(getfilename(files[0]),groundTruth)


In [15]:
scene = Config.Scene()
scene.add(groundTruth.mesh)
pyrender.Viewer(scene,render_flags={"all_solid":True},viewer_flags={"use_direct_lighting":True})

<pyglet.libs.x11.xlib.LP_struct__XDisplay object at 0x7fb1341ab510> 482 0 0 640 480 0 24 1 <pyglet.libs.x11.xlib.LP_struct_anon_18 object at 0x7fb104955158> 8210 <cparam 'P' (0x7fb10494d650)>


Viewer(width=640, height=480)

In [17]:
CameraConfiguration.set_camera("dragon_wing",groundTruth)

AttributeError: 'CameraConfiguration' object has no attribute 'GetScene'

In [ ]:
import pyglet


In [ ]:
window = pyglet.window.Window(display=None)
window.on_close = lambda:window.close()
label = pyglet.text.Label('Hello, world',
                          font_name='Times New Roman',
                          font_size=36,
                          x=window.width//2, y=window.height//2,
                          anchor_x='center', anchor_y='center')

In [12]:
window.close()

In [18]:
import tabulate

In [50]:
example_data = {"":["val1","val2","val3"],"row1":[1,2,3],"row2":[3,4,5]}
print(tabulate.tabulate(example_data,headers="keys"))

        row1    row2
----  ------  ------
val1       1       3
val2       2       4
val3       3       5


In [63]:
example_data =[["test1","test2","test3"],["test1","test","test3"],["test1","test","test3"]]
print(tabulate.tabulate(example_data,headers=["--","a","b"],tablefmt="github"))

| --    | a     | b     |
|-------|-------|-------|
| test1 | test2 | test3 |
| test1 | test  | test3 |
| test1 | test  | test3 |
